This version of solving digit recognition problem using MNIST dataset is implemented using keras: simple neural network and colvolutional neural network

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.datasets import mnist
from tensorflow.python.keras.models import Model, Sequential
from tensorflow.python.keras.layers import Input, Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.python.keras import utils

np.random.seed(1)

## Load data

The data files train.csv and test.csv contain gray-scale images of hand-drawn digits, from zero through nine.

The training data set, (train.csv), has 785 columns. The first column, called "label", is the digit that was drawn by the user. The rest of the columns contain the pixel-values of the associated image. Each image is 28 pixels in height and 28 pixels in width, for a total of 784 pixels in total. Each pixel has a single pixel-value associated with it, indicating the lightness or darkness of that pixel, with higher numbers meaning darker. This pixel-value is an integer between 0 and 255, inclusive.

Each pixel column in the training set has a name like pixelx, where x is an integer between 0 and 783, inclusive. To locate this pixel on the image, suppose that we have decomposed x as x = i * 28 + j, where i and j are integers between 0 and 27, inclusive. Then pixelx is located on row i and column j of a 28 x 28 matrix, (indexing by zero).


In [2]:
train = pd.read_csv('../data/digit-recognizer/train.csv')
test = pd.read_csv('../data/digit-recognizer/test.csv')

## Data preprocessing

Data preprocessing would include:
* Normalize the data (numeric values are rescaled in order to have values between 0 and 1)
* Split the dataset on train and validation datasets
* Represent target value as zero vector with the length equals to the number of categories and "1" value on a position corresponding to a target value, i.e. target value "4" will be threated as [0, 0, 0, 0, 1, 0, 0, 0, 0, 0] vector. This is an important step because predicting "7" instead of real target value "8" is not better than predicting "0", both predictions are wrong

In [3]:
test_normalized = test.values.astype('float32')
test_normalized /= 255

X = train.iloc[:, 1:].values.astype('float32')
X /= 255
y = utils.to_categorical(train.iloc[:, 0])

## Build a model


### Simple multi-layer neural network


In [9]:
num_pixels = 784
batch_size = 64
epochs = 20
hidden_layer_size = 32
num_classes = 10
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=5)

def create_simple_nn_model():
    model = Sequential()
    model.add(Dense(hidden_layer_size, input_dim=num_pixels, kernel_initializer='normal', activation='relu'))
    model.add(Dense(num_classes, kernel_initializer='normal', activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = create_simple_nn_model()
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=batch_size, verbose=2)
scores = model.evaluate(X_test, y_test, verbose=0)
print(f"Simple NN prediction error: {100-scores[1]*100:.2f}%")

Train on 37800 samples, validate on 4200 samples
Epoch 1/20
 - 3s - loss: 6.8504 - acc: 0.4727 - val_loss: 4.1784 - val_acc: 0.6407
Epoch 2/20
 - 3s - loss: 2.9171 - acc: 0.7099 - val_loss: 1.9541 - val_acc: 0.7576
Epoch 3/20
 - 3s - loss: 1.5870 - acc: 0.7919 - val_loss: 1.3228 - val_acc: 0.8014
Epoch 4/20
 - 3s - loss: 1.1496 - acc: 0.8197 - val_loss: 1.0287 - val_acc: 0.8226
Epoch 5/20
 - 3s - loss: 0.8940 - acc: 0.8356 - val_loss: 0.8420 - val_acc: 0.8357
Epoch 6/20
 - 2s - loss: 0.7314 - acc: 0.8503 - val_loss: 0.7180 - val_acc: 0.8467
Epoch 7/20
 - 3s - loss: 0.6224 - acc: 0.8631 - val_loss: 0.6306 - val_acc: 0.8605
Epoch 8/20
 - 3s - loss: 0.5423 - acc: 0.8754 - val_loss: 0.5588 - val_acc: 0.8693
Epoch 9/20
 - 3s - loss: 0.4783 - acc: 0.8843 - val_loss: 0.5098 - val_acc: 0.8800
Epoch 10/20
 - 3s - loss: 0.4291 - acc: 0.8935 - val_loss: 0.4629 - val_acc: 0.8881
Epoch 11/20
 - 3s - loss: 0.3867 - acc: 0.9016 - val_loss: 0.4295 - val_acc: 0.8960
Epoch 12/20
 - 3s - loss: 0.3506 - a

### Simple convolutional neural network


In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=5)
# reshape in order to have [samples][width][height][channels] shape
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1).astype('float32')

In [18]:
batch_size = 200
epochs = 10

def create_conv_nn_model():
    model = Sequential()
    model.add(Conv2D(32, (5, 5), input_shape=(28, 28, 1), activation='relu'))
    model.add(MaxPooling2D())
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

cnn_model = create_conv_nn_model()
cnn_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=batch_size)
scores = cnn_model.evaluate(X_test, y_test, verbose=0)
print(f"Convolutional NN prediction error: {100-scores[1]*100:.2f}%")

Train on 37800 samples, validate on 4200 samples
Epoch 1/10
37800/37800 [==============================]37800/37800 [==============================] - 48s 1ms/step - loss: 0.2962 - acc: 0.9165 - val_loss: 0.1001 - val_acc: 0.9688

Epoch 2/10
37800/37800 [==============================]37800/37800 [==============================] - 45s 1ms/step - loss: 0.0865 - acc: 0.9744 - val_loss: 0.0703 - val_acc: 0.9800

Epoch 3/10
37800/37800 [==============================]37800/37800 [==============================] - 45s 1ms/step - loss: 0.0595 - acc: 0.9817 - val_loss: 0.0619 - val_acc: 0.9807

Epoch 4/10
37800/37800 [==============================]37800/37800 [==============================] - 45s 1ms/step - loss: 0.0461 - acc: 0.9858 - val_loss: 0.0607 - val_acc: 0.9788

Epoch 5/10
37800/37800 [==============================]37800/37800 [==============================] - 46s 1ms/step - loss: 0.0382 - acc: 0.9882 - val_loss: 0.0496 - val_acc: 0.9843

Epoch 6/10
37800/37800 [=================

## Submit the prediction

In [16]:
prediction = []
output = model.predict(test_normalized)
prediction = np.argmax(output, axis=1)
submission = pd.DataFrame({
    "ImageId": list(range(1, test.shape[0]+1)),
    "Label": prediction
})
submission.to_csv("../data/digit-recognizer/digit-recognizer-submission.csv", index=False)
submission.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,4
4,5,3
